In [ ]:
import requests

headers = {
    "User-Agent": "Mozilla/5.0 (DataScience Project)"
}

url = "https://api.chess.com/pub/player/Sreizzzy"
r = requests.get(url, headers=headers)

print(r.json())


In [13]:
import berserk

def load_token(path):
    variable = {}
    with open(path, "r") as f:
        for line in f:
            line = line.strip()

            # skip baris kosong & komentar
            if not line or line.startswith("#"):
                continue

            key, value = line.split("=", 1)
            variable[key.strip()] = value.strip()[1:-1]

    return variable

variable = load_token("../secret.txt")
TOKEN = variable["TOKEN"]

session = berserk.TokenSession(TOKEN)
client = berserk.Client(session=session)

client.account.get()

{'id': 'gagah_adilaga',
 'username': 'Gagah_Adilaga',
 'perfs': {'bullet': {'games': 144, 'rating': 1583, 'rd': 52, 'prog': 4},
  'blitz': {'games': 676, 'rating': 1612, 'rd': 45, 'prog': -15},
  'rapid': {'games': 17, 'rating': 1735, 'rd': 304, 'prog': 4, 'prov': True},
  'classical': {'games': 0,
   'rating': 1500,
   'rd': 500,
   'prog': 0,
   'prov': True},
  'correspondence': {'games': 0,
   'rating': 1500,
   'rd': 500,
   'prog': 0,
   'prov': True},
  'puzzle': {'games': 221, 'rating': 1909, 'rd': 89, 'prog': 0}},
 'createdAt': datetime.datetime(2021, 9, 21, 2, 6, 16, 456000, tzinfo=datetime.timezone.utc),
 'seenAt': datetime.datetime(2026, 2, 8, 5, 58, 0, 774000, tzinfo=datetime.timezone.utc),
 'playTime': {'total': 269516, 'tv': 0},
 'url': 'https://lichess.org/@/Gagah_Adilaga',
 'count': {'all': 844,
  'rated': 837,
  'draw': 26,
  'loss': 399,
  'win': 419,
  'bookmark': 0,
  'playing': 0,
  'import': 0,
  'me': 0},
 'followable': True,
 'following': False,
 'blocking': Fa

In [1]:
import zstandard as zstd
import chess.pgn
import io

file_path = "../lichess_db_standard_rated_2025-12.pgn.zst"

with open(file_path, "rb") as f:
    dctx = zstd.ZstdDecompressor()
    stream_reader = dctx.stream_reader(f)
    text_stream = io.TextIOWrapper(stream_reader, encoding="utf-8")

    while True:
        game = chess.pgn.read_game(text_stream)
        if game is None:
            break
        
        # akses header
        headers = game.headers
        print(headers["White"], headers["Black"], headers["Result"])
        break  # hapus ini untuk looping semua


Izamutdin Raximov_Toshpolat 0-1


In [8]:
import zstandard as zstd
import chess.pgn
import io

file_path = "../lichess_db_standard_rated_2025-12.pgn.zst"

with open(file_path, "rb") as f:
    dctx = zstd.ZstdDecompressor()
    stream = dctx.stream_reader(f)
    text_stream = io.TextIOWrapper(stream, encoding="utf-8")

    for i in range(5):
        game = chess.pgn.read_game(text_stream)
        if game is None:
            break

        print(f"\n===== GAME {i+1} =====")
        print(game)



===== GAME 1 =====
[Event "Rated Blitz game"]
[Site "https://lichess.org/8LwiNwZm"]
[Date "2025.12.01"]
[Round "-"]
[White "Izamutdin"]
[Black "Raximov_Toshpolat"]
[Result "0-1"]
[UTCDate "2025.12.01"]
[UTCTime "00:00:13"]
[WhiteElo "1811"]
[BlackElo "1831"]
[WhiteRatingDiff "-6"]
[BlackRatingDiff "+5"]
[ECO "A40"]
[Opening "Queen's Pawn Game"]
[TimeControl "300+0"]
[Termination "Normal"]

1. d4 { [%clk 0:05:00] } 1... c6 { [%clk 0:05:00] } 2. e3 { [%clk 0:05:00] } 2... e6 { [%clk 0:04:59] } 3. c3 { [%clk 0:04:59] } 3... d5 { [%clk 0:04:59] } 4. Bd3 { [%clk 0:04:59] } 4... c5 { [%clk 0:04:57] } 5. Nf3 { [%clk 0:04:59] } 5... c4 { [%clk 0:04:56] } 6. Bc2 { [%clk 0:04:58] } 6... b5 { [%clk 0:04:55] } 7. Nbd2 { [%clk 0:04:57] } 7... a5 { [%clk 0:04:53] } 8. O-O { [%clk 0:04:55] } 8... b4 { [%clk 0:04:47] } 9. Ne5 { [%clk 0:04:54] } 9... f6 { [%clk 0:04:40] } 10. Qh5+ { [%clk 0:04:53] } 10... Ke7 { [%clk 0:04:27] } 11. Nf7 { [%clk 0:04:23] } 11... Qe8 { [%clk 0:04:23] } 12. Bxh7 { [%clk 0

In [1]:
def process_game(game):
    if game is None:
        return "STOP"

    headers = game.headers

    # --- basic validation ---
    try:
        white_elo = int(headers.get("WhiteElo", 0))
        black_elo = int(headers.get("BlackElo", 0))
    except ValueError:
        return None

    # is_rapid = headers.get("Event") == "Rated Rapid game"
    white_title = headers.get("WhiteTitle")
    black_title = headers.get("BlackTitle")

    # if white_title != "GM" and black_title != "GM":
    #     return None

    # --- filter rating ---
    if white_elo < 2000 or black_elo < 2000:
        return None

    # --- filter time control (rapid) ---
    time_control = headers.get("TimeControl", "")
    base_time = int(time_control.split("+")[0]) if "+" in time_control else 0
    if not (600 <= base_time <= 1800):  # rapid lichess
        return None

    # --- collect moves ---
    moves = [m.move.uci() for m in game.mainline()]

    return {
        "link": headers.get("Site"),
        "white": headers.get("White"),
        "black": headers.get("Black"),
        "white_titled": white_title,
        "black_titled": black_title,
        "white_elo": white_elo,
        "black_elo": black_elo,
        "result": headers.get("Result"),
        "white_rating_diff": headers.get("WhiteRatingDiff"),
        "black_rating_diff": headers.get("BlackRatingDiff"),
        "time_control": time_control,
        "termination": headers.get("Termination"),
        "eco": headers.get("ECO"),
        "opening": headers.get("Opening"),
        "moves_uci": " ".join(moves),
        "num_moves": len(moves)
    }

In [2]:
import zstandard as zstd
import chess.pgn
import io
import pandas as pd

file_path = "../lichess_db_standard_rated_2025-12.pgn.zst"

rows = []
MAX_GAMES = 20

with open(file_path, "rb") as f:
    dctx = zstd.ZstdDecompressor()
    stream = dctx.stream_reader(f)
    text_stream = io.TextIOWrapper(stream, encoding="utf-8")

    while len(rows) < MAX_GAMES:
        game = chess.pgn.read_game(text_stream)
        result = process_game(game)
        
        if result == "STOP":
            break
        
        if result is not None:
            rows.append(result)


df = pd.DataFrame(rows)

print(df.head())
print(f"\nTotal games collected: {len(df)}")

                           link             white            black  \
0  https://lichess.org/SVd8eyTK        Kinga_2602         tunoperu   
1  https://lichess.org/pdlJtWXn  Pradigintungkbpl      brisachess1   
2  https://lichess.org/KUhNmo4h     No_Limitation       flandeluxe   
3  https://lichess.org/kbDjhp9o        HIKARULSEN  Juniorsolorzano   
4  https://lichess.org/D0QgwH18         Dralsaadi         mohantaj   

  white_titled black_titled  white_elo  black_elo result white_rating_diff  \
0         None         None       2046       2101    0-1                -5   
1         None         None       2120       2081    1-0                +5   
2         None         None       2066       2076    0-1                -6   
3         None         None       2030       2015    0-1                -6   
4         None         None       2092       2083    1-0                +6   

  black_rating_diff time_control   termination  eco  \
0                +5        600+0  Time forfeit  B03   


In [3]:
df["moves_uci"][0]

'e2e4 g8f6 e4e5 f6d5 d2d4 d7d6 c2c4 d5b6 f2f4 c8f5 f1d3 f5d3 d1d3 e7e6 g1f3 b8c6 e1g1 f8e7 b1c3 e8g8 a2a3 c6a5 f3d2 d6d5 c4c5 b6c4 b2b4 c4d2 c1d2 a5c4 f4f5 e6f5 f1f5 c7c6 a1f1 b7b6 d2c1 a7a5 d3g3 a5b4 c1h6 g7g6 h6f8 d8f8 e5e6 f7f6 a3b4 g8h8 f5f3 c4d2 c5b6 d2f1 f3f1 e7b4 b6b7 a8e8 f1f6 f8f6 b7b8q e8b8 g3b8 f6f8 b8e5 h8g8 c3d1 b4d6 e5e2 f8f6 e2e3 g8g7 d1f2 f6f4 e3f4 d6f4 f2d3 f4e3 g1f1 e3d4 f1e2 g7f6 e2f3 f6e6 h2h4 d4f6 d3f4 e6f5 h4h5 g6g5'

In [4]:
import zstandard as zstd
import chess.pgn
import io
import os
from tqdm import tqdm
import pandas as pd

file_path = "../lichess_db_standard_rated_2025-12.pgn.zst"

TOTAL_GAMES = 94_847_276
TOTAL_CSV = 100_000
BATCH_SIZE = 2_000
CSV_FILE = "lichess_rapid_elo2000.csv"

rows = []
collected_count = 0

with open(file_path, "rb") as f:
    dctx = zstd.ZstdDecompressor()
    stream = dctx.stream_reader(f)
    text_stream = io.TextIOWrapper(stream, encoding="utf-8")

    with tqdm(total=TOTAL_GAMES, desc="Processing PGN", smoothing=0.05, miniters=100) as pbar:

        while True:
            game = chess.pgn.read_game(text_stream)
            result = process_game(game)
        
            if game is None or result == "STOP":
                print("Game is None")
                break

            if result is not None:
                rows.append(result)
                collected_count += 1

            if pbar.n % 500 == 0:
                pbar.set_postfix(collected=collected_count)

            # save checkpoint
            if len(rows) >= BATCH_SIZE:
                pd.DataFrame(rows).to_csv(
                    CSV_FILE,
                    mode="a",
                    header=not os.path.exists(CSV_FILE),
                    index=False
                )
                rows.clear()

            if collected_count >= TOTAL_CSV:
                print("Target CSV rows reached.")
                break

            pbar.update(1)


df = pd.read_csv(CSV_FILE)
print(df.head())
print(f"\nTotal games collected: {len(df)}")


Processing PGN:   0%|          | 5157/94847276 [00:15<79:06:13, 333.04it/s, collected=33]


KeyboardInterrupt: 

In [18]:
import zstandard as zstd
import chess.pgn
import io
import os
from tqdm import tqdm
import pandas as pd

file_path = "../lichess_db_standard_rated_2025-12.pgn.zst"

TOTAL_GAMES = 94_847_276
TOTAL_CSV = 100_000
BATCH_SIZE = 2_000
CHECKPOINT_FILE = "checkpoint.txt"
CSV_FILE = "lichess_rapid_elo2000.csv"

rows = []
collected_count = 0

start_game = 0
if os.path.exists(CHECKPOINT_FILE):
    with open(CHECKPOINT_FILE) as f:
        start_game = int(f.read())

# Kalau CSV sudah ada, sinkronkan collected_count
if os.path.exists(CSV_FILE):
    collected_count = sum(1 for _ in open(CSV_FILE)) - 1  # minus header

print(f"Resume from game index: {start_game}")
print(f"Already collected rows  : {collected_count}")

with open(file_path, "rb") as f:
    dctx = zstd.ZstdDecompressor()
    stream = dctx.stream_reader(f)
    text_stream = io.TextIOWrapper(stream, encoding="utf-8")

    skipped = 0
    while skipped < start_game:
        game = chess.pgn.read_game(text_stream)
        if game is None:
            break
        skipped += 1

    with tqdm(total=TOTAL_GAMES, initial=start_game, desc="Processing PGN", smoothing=0.05, miniters=100) as pbar:

        while True:
            game = chess.pgn.read_game(text_stream)
            result = process_game(game)
        
            if game is None or result == "STOP":
                print("Game is None")
                break

            if result is not None:
                rows.append(result)
                collected_count += 1

            if pbar.n % 500 == 0:
                pbar.set_postfix(collected=collected_count)

            # save checkpoint
            if len(rows) >= BATCH_SIZE:
                pd.DataFrame(rows).to_csv(
                    CSV_FILE,
                    mode="a",
                    header=not os.path.exists(CSV_FILE),
                    index=False
                )
                rows.clear()

                with open(CHECKPOINT_FILE, "w") as f_cp:
                    f_cp.write(str(pbar.n))

            if collected_count >= TOTAL_CSV:
                print("Target CSV rows reached.")
                break

            pbar.update(1)


df = pd.read_csv(CSV_FILE)
print(df.head())
print(f"\nTotal games collected: {len(df)}")


Resume from game index: 270122
Already collected rows  : 2000


KeyboardInterrupt: 

Resume from game index: 0
Already collected rows  : 0
Processing PGN:   0%|          | 288494/94847276 [14:47<80:48:43, 325.03it/s, collected=2149]

In [16]:
df = pd.read_csv(CSV_FILE)
print(df.head())
print(f"\nTotal games collected: {len(df)}")

                           link             white            black  \
0  https://lichess.org/SVd8eyTK        Kinga_2602         tunoperu   
1  https://lichess.org/pdlJtWXn  Pradigintungkbpl      brisachess1   
2  https://lichess.org/KUhNmo4h     No_Limitation       flandeluxe   
3  https://lichess.org/kbDjhp9o        HIKARULSEN  Juniorsolorzano   
4  https://lichess.org/D0QgwH18         Dralsaadi         mohantaj   

  white_titled black_titled  white_elo  black_elo result  white_rating_diff  \
0          NaN          NaN       2046       2101    0-1               -5.0   
1          NaN          NaN       2120       2081    1-0                5.0   
2          NaN          NaN       2066       2076    0-1               -6.0   
3          NaN          NaN       2030       2015    0-1               -6.0   
4          NaN          NaN       2092       2083    1-0                6.0   

   black_rating_diff time_control   termination  eco  \
0                5.0        600+0  Time forfeit 